 ## *Text-to-3D Game Asset Generator*
Building an AI Pipeline for Game Development
This notebook demonstrates a proof-of-concept for generating 3D game assets from text prompts using OpenAI's Shap-E model. The pipeline includes:

Text-to-3D generation

Parameter experimentation

Output optimization for games

Preparation for API deployment

Project for Red Panda Games ML/LLM Intern Role

In [1]:
# Install all required
!pip install diffusers transformers torch accelerate trimesh pygltflib
!pip install git+https://github.com/openai/shap-e.git
!pip install fastapi uvicorn python-multipart aiofiles



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 728.5/728.5 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.1 MB/s eta 0:00:00
  Cloning https://github.com/openai/shap-e.git to /tmp/pip-req-build-fa892aj9
  Running command git clone --filter=blob:none --quiet https://github.com/openai/shap-e.git /tmp/pip-req-build-fa892aj9
  Resolved https://github.com/openai/shap-e.git to commit 50131012ee11c9d2617f3886c10f000d3c7a3b43
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-install-0yqfckcp/clip_e88b5a08789f4ae48034889f53fceb15
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-install-0yqfckcp/clip_e88b5a08789f4ae48034889f53fceb15
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.9/115.9 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━

### Load Shap-E Models

Load the transmitter and text model along with diffusion configuration.

In [2]:
# Import libraries
import torch
import numpy as np
import trimesh
import os
from shap_e.diffusion.sample import sample_latents
from shap_e.diffusion.gaussian_diffusion import diffusion_from_config
from shap_e.models.download import load_model, load_config
from shap_e.util.notebooks import decode_latent_mesh
from google.colab import files

In [3]:
# Initialize device and load models
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")



Using device: cuda


In [4]:
# Load Shap-E models
xm = load_model('transmitter', device=device)
model = load_model('text300M', device=device)
diffusion = diffusion_from_config(load_config('diffusion'))
print("✅ Models loaded successfully!")

/usr/local/lib/python3.12/dist-packages/shap_e/models/nn/checkpoint.py:31: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd
/usr/local/lib/python3.12/dist-packages/shap_e/models/nn/checkpoint.py:43: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @custom_bwd
/usr/local/lib/python3.12/dist-packages/shap_e/models/nn/checkpoint.py:61: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd
/usr/local/lib/python3.12/dist-packages/shap_e/models/nn/checkpoint.py:86: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @custom_bwd


  0%|          | 0.00/1.78G [00:00<?, ?iB/s]

100%|███████████████████████████████████████| 890M/890M [00:11<00:00, 82.7MiB/s]


  0%|          | 0.00/1.26G [00:00<?, ?iB/s]

✅ Models loaded successfully!


### Generate 3D Asset

Function to generate a 3D mesh from a text prompt.

In [5]:
def gen_3d(prompt, scale=12.0, steps=32):  # ← Changed defaults here
    """
    Generate 3D asset from text
    """
    print(f"🎨 Generating: '{prompt}'...")

    latents = sample_latents(
        batch_size=1,
        model=model,
        diffusion=diffusion,
        guidance_scale=scale,
        model_kwargs=dict(texts=[prompt]),
        progress=True,
        clip_denoised=True,
        use_fp16=True,
        use_karras=True,
        karras_steps=steps,
        sigma_min=1e-3,
        sigma_max=160,
        s_churn=0,
    )

    with torch.no_grad():
        mesh = decode_latent_mesh(xm, latents[0]).tri_mesh()

    print(f"✅ Done! V: {len(mesh.verts)}, F: {len(mesh.faces)}")
    return mesh

### Optimize Mesh

Reduce the number of faces to make the mesh game-ready.

In [6]:
def optimize(mesh, target=1500):
    """
    Alternative approach using face count directly
    """
    vertices = np.array(mesh.verts)
    faces = np.array(mesh.faces)
    org_mesh = trimesh.Trimesh(vertices=vertices, faces=faces)

    org_faces = len(org_mesh.faces)
    print(f"📊 Org: {org_faces} triangles")

    # Use the method that actually works with face count
    try:
        # Try the quadric decimation with face count
        reduce_mesh = org_mesh.simplify_quadric_decimation(target)
    except:
        # Fallback: use the ratio approach
        target_ratio = target / org_faces
        reduce_mesh = org_mesh.simplify_quadric_decimation(target_ratio)

    reduction = 100 * (1 - len(reduce_mesh.faces)/org_faces)
    print(f"⚡ Reduced: {len(reduce_mesh.faces)} triangles (-{reduction:.1f}%)")

    return reduce_mesh

### Save Mesh

Export the mesh as .obj and .glb.

In [7]:
def save(mesh, fname):
    """
    Save 3D mesh - FIXED VERSION
    """
    os.makedirs('outputs', exist_ok=True)

    if hasattr(mesh, 'write_obj'):

        obj_file = f'outputs/{fname}.obj'
        with open(obj_file, 'w') as f:
            mesh.write_obj(f)
    else:

        obj_file = f'outputs/{fname}.obj'
        mesh.export(obj_file)

    # Load and convert to GLB
    mesh_obj = trimesh.load(obj_file)
    glb_file = f'outputs/{fname}.glb'
    mesh_obj.export(glb_file)

    print(f"💾 Saved: {glb_file}")
    return glb_file

### Parameter Experiments

Test different guidance scales and steps to find optimal parameters.

In [8]:
def parm_exp():
    """
    Test parameters for best results
    """
    results = []
    base_prompt = "a low-poly tree"

    print("🔬 Testing parameters...")

    for scale in [7.0, 12.0, 15.0, 18.0, 25.0]:
        mesh = gen_3d(base_prompt, scale=scale, steps=64)
        results.append({
            'param': 'scale',
            'value': scale,
            'verts': len(mesh.verts),
            'faces': len(mesh.faces)
        })

    for steps in [32, 64, 128]:
        mesh = gen_3d(base_prompt, scale=15.0, steps=steps)
        results.append({
            'param': 'steps',
            'value': steps,
            'verts': len(mesh.verts),
            'faces': len(mesh.faces)
        })

    return results

# Run experiments
exp_results = parm_exp()

best_scale = 12.0
best_steps = 32

print(f" Optimal parameters found: Scale={best_scale}, Steps={best_steps}")

🔬 Testing parameters...
🎨 Generating: 'a low-poly tree'...


  0%|          | 0/64 [00:00<?, ?it/s]

/usr/local/lib/python3.12/dist-packages/shap_e/models/stf/renderer.py:286: UserWarning: exception rendering with PyTorch3D: No module named 'pytorch3d'
  warnings.warn(f"exception rendering with PyTorch3D: {exc}")
/usr/local/lib/python3.12/dist-packages/shap_e/models/stf/renderer.py:287: UserWarning: falling back on native PyTorch renderer, which does not support full gradients
  warnings.warn(


✅ Done! V: 42380, F: 84752
🎨 Generating: 'a low-poly tree'...


  0%|          | 0/64 [00:00<?, ?it/s]

...

✅ Done! V: 39106, F: 78196
🎨 Generating: 'a low-poly tree'...


  0%|          | 0/64 [00:00<?, ?it/s]

✅ Done! V: 53376, F: 106760
🎨 Generating: 'a low-poly tree'...


  0%|          | 0/128 [00:00<?, ?it/s]

✅ Done! V: 46572, F: 93148
 Optimal parameters found: Scale=12.0, Steps=32


In [9]:
# Install the required simplification library
!pip install fast-simplification

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 27.1 MB/s eta 0:00:00


### Test Prompts

Generate and optimize 3D trees with multiple variations.

In [10]:
def test():
    """
    Test different prompts
    """
    variation = [
        "a low-poly tree with less than 1000 polygons",
        "a very simple tree model for mobile game",
        "a minimalist tree asset with low polygon count",
        "a game-ready low-poly tree asset",
        "a tree 3d model with optimized geometry"
    ]

    results = []
    for prompt in variation:
        mesh = gen_3d(prompt, best_scale, best_steps)
        optimized = optimize(mesh, 1500)
        glb_file = save(optimized, f"opt_{prompt[:10]}")
        files.download(glb_file)
        results.append({'prompt': prompt, 'faces': len(mesh.faces)})

    return results

prompt_results = test()

🎨 Generating: 'a low-poly tree with less than 1000 polygons'...


  0%|          | 0/32 [00:00<?, ?it/s]

✅ Done! V: 49552, F: 99128
📊 Org: 99128 triangles
⚡ Reduced: 97628 triangles (-1.5%)
💾 Saved: outputs/opt_a low-poly.glb


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

🎨 Generating: 'a tree 3d model with optimized geometry'...


  0%|          | 0/32 [00:00<?, ?it/s]

✅ Done! V: 23386, F: 46416
📊 Org: 46416 triangles
⚡ Reduced: 44916 triangles (-3.2%)
💾 Saved: outputs/opt_a tree 3d .glb


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Generate Portfolio

Generate multiple game-ready assets like sword, shield, and helmet.

In [11]:
def gen_portfolio():
    """
    Generate multiple assets
    """
    portfolio = [
        ("a low-poly fantasy sword", "sword"),
        ("a magical health potion bottle", "potion"),
        ("a medieval wooden shield", "shield"),
        ("a fantasy character helmet", "helmet"),
        ("a simple rock obstacle", "rock")
    ]

    print("🎮 Generating portfolio...")

    for prompt, name in portfolio:
        print(f"\n⭐ Generating {name}...")
        mesh = gen_3d(prompt, best_scale, best_steps)
        optimized = optimize(mesh, 2000)
        glb_file = save(optimized, f"game_{name}")
        files.download(glb_file)

gen_portfolio()

🎮 Generating portfolio...

⭐ Generating sword...
🎨 Generating: 'a low-poly fantasy sword'...


  0%|          | 0/32 [00:00<?, ?it/s]

✅ Done! V: 29838, F: 59652
📊 Org: 59652 triangles
⚡ Reduced: 57652 triangles (-3.4%)
💾 Saved: outputs/game_sword.glb


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...


⭐ Generating rock...
🎨 Generating: 'a simple rock obstacle'...


  0%|          | 0/32 [00:00<?, ?it/s]

✅ Done! V: 60480, F: 120984
📊 Org: 120984 triangles
⚡ Reduced: 118984 triangles (-1.7%)
💾 Saved: outputs/game_rock.glb


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>